
# BMIS-2542: Data Programming with Python 
##### Katz Graduate School of Business, Fall 2019

## Session 9: Predictive Models II - Decision Trees
***

In this notebook, we continue with our discussion on supervised learning approaches and describe tree-based methods for classification and regression. <br>
The book ["An Introduction to Statistical Learning"](http://www-bcf.usc.edu/~gareth/ISL/) (chapter 8 on decision trees) has good reference material on the topics we discuss today. <br>
Also, the chapter 6 and chapter 7 of the book [Hands on Machine Learning with Scikit-Learn and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291) are helpful too.

### Additional Software 
* We need to install the *graphviz* and *pydotplus* libraries to visualize decision trees.
* First, install *graphviz*, as per the instructions given [here](http://www.graphviz.org/download/). Windows users can get the stable releases [here](https://graphviz.gitlab.io/_pages/Download/Download_windows.html).<br>
  For *Mac OS*, this might be the best option: Follow the instructions given [here](http://macappstore.org/graphviz-2/).<br>
  If the command in step 3 gives an error, try the command > `brew install graphviz`.<br>
* For *Windows*, you may need to add to the system *PATH* variable the location of the folders where anaconda and graphviz are installed: (e.g., C:\Program Files (x86)\Graphviz2.38\bin, and C:\Program Files\Anaconda3\Scripts).

* Once graphviz is installed, go to Anaconda prompt (Windows) or Terminal (Mac OS) and issue the following commands one by one:
  - `conda install graphviz`
  - `conda install python-graphviz`
  - `conda install pydotplus`
* you may have to restart your computer and Jupyter again.

In [ ]:
from IPython.display import Image
import graphviz
import pydotplus 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import pydotplus as pplus
import graphviz
from IPython.display import Image

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
pd.set_option('display.notebook_repr_html', True)
%matplotlib inline
plt.style.use('seaborn-white')

C:\Users\Sunny\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
#This function draws decision trees using the pydotplus and graphviz libraries
def print_tree(estimator, features, class_names=None, filled=True):
    tree = estimator
    names = features
    color = filled
    classn = class_names
    
    dot_data = StringIO()
    export_graphviz(estimator, out_file=dot_data, feature_names=features, class_names=classn, filled=filled)
    graph = pplus.graph_from_dot_data(dot_data.getvalue())
    return(graph)

## Decision Trees
`Decision Trees` are versatile machine learning (ML) algorithms that can be used for both classification and regression tasks and are capable of fitting complex datasets. Also, they are the fundamental components of `Random Forests`, which are among the most powerful ML algorithms available today.

Decision Trees:
 - are simple to understand and to interpret. Trees can be visualized
 - requires little data preparation
 - can handle multi-output problems
 - may create over-complex trees that do not generalise the data well
 - can be unstable because small variations in the data might result in a completely different tree being generated

### Classification Trees
Classification trees employ decision trees to address classification problems.<br> `Scikit-Learn` uses the `CART (Classification and Regression Tree)` algorithm to produce decision trees, using `recursive binary splitting`. 
Let's use the Iris dataset to learn the basic concepts of Decision Trees.

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
type(iris)

sklearn.utils.Bunch

In [4]:
print(iris.feature_names)
print(iris.target_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [5]:
X = iris.data[:,2:] # petal length and width
y = iris.target

In [6]:
tree_clf = DecisionTreeClassifier(max_depth=2,random_state=23)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=23, splitter='best')

In [7]:
graphClf = print_tree(tree_clf, features=iris.feature_names[2:], class_names=iris.target_names)
Image(graphClf.create_png())

InvocationException: GraphViz's executables not found

A node's `gini` attribute measures its impurity: a node is **pure** (`gini=0`) if all training instances it applies to belong to the same class. For example, the `depth-1` left node applies to only `Iris-Setosa` training instances, hence it is pure and its gini score is 0.<br>
Gini impurity is given by: 
<br> $${G}_i = 1 - \sum_{k=1}^{n} {{p_i}_k}^2 $$<br>
where ${p_i}_k$ is the ratio of class `k` instances among the training instances in the $i^{th}$ node.

Let's calculate the `gini score` of the `depth-2` left node.

In [8]:
1 - (49/54)**2 - (5/54)**2

0.1680384087791495

#### Estimating Class Probabilities
For  a classification tree, a given observation is predicted to be in the *most commonly occuring class* of training observations in the region to which it belongs. 

In [9]:
tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [10]:
tree_clf.predict([[5,1.5]])

array([1])

#### Other Impurity Measures
Another measure of node impurity is `entropy`. A node's entropy is `0` if it contains instances of only one class. 
The entropy of the $i^{th}$ node is given by:
$${H}_i = - \sum_{k=1}^{n} {p_i}_k log({p_i}_k) $$ where ${p_i}_k$ is the ratio of class `k` instances among the training instances in the $i^{th}$ node and ${p_i}_k\neq0$.

#### <mark> Exercise </mark>
(a) Calculate the `entropy` of the `depth-2` left node (Use `np.log2` to get the logarithm).<br>
(b) Generate a new decision tree using `entropy` as the measure of impurity. You need to set the `criterion` parameter equal to `entropy` when calling the `DecisionTreeClassifier`. <br>Visualize the decision tree and compare it to the tree generated with `gini` score.

### Heart Disease Dataset

Let's do another example using the heart disease dataset, `Heart.csv`.<br>Read more about this dataset [here](https://archive.ics.uci.edu/ml/datasets/Heart+Disease).

In [11]:
heart = pd.read_csv('Heart.csv').drop('Unnamed: 0', axis=1).dropna()
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [12]:
# We can use pandas factorize function to convert non-numeric categorical variables into factors
labels, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
print(labels)
print(uniques)

[0 0 1 2 0]
['b' 'a' 'c']


In [13]:
print(heart['ChestPain'].unique())
print(heart['Thal'].unique())
print(heart['AHD'].unique())

['typical' 'asymptomatic' 'nonanginal' 'nontypical']
['fixed' 'normal' 'reversable']
['No' 'Yes']


In [14]:
# Factorize the categorical variables
heart.ChestPain = pd.factorize(heart.ChestPain)[0]
heart.Thal = pd.factorize(heart.Thal)[0]
heart.AHD=pd.factorize(heart.AHD)[0]

heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,0,145,233,1,2,150,0,2.3,3,0.0,0,0
1,67,1,1,160,286,0,2,108,1,1.5,2,3.0,1,1
2,67,1,1,120,229,0,2,129,1,2.6,2,2.0,2,1
3,37,1,2,130,250,0,0,187,0,3.5,3,0.0,1,0
4,41,0,3,130,204,0,2,172,0,1.4,1,0.0,1,0


In [15]:
# prepare data for decision tree classifier
X = heart.drop('AHD', axis=1) 
y = heart.AHD

X.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal
0,63,1,0,145,233,1,2,150,0,2.3,3,0.0,0
1,67,1,1,160,286,0,2,108,1,1.5,2,3.0,1
2,67,1,1,120,229,0,2,129,1,2.6,2,2.0,2
3,37,1,2,130,250,0,0,187,0,3.5,3,0.0,1
4,41,0,3,130,204,0,2,172,0,1.4,1,0.0,1


In [16]:
#split to training and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=23)
print(len(X_train))
print(len(X_test))

148
149


In [17]:
# documentation: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
clftree = DecisionTreeClassifier(max_depth=None, max_leaf_nodes=6, max_features=3, random_state=23)
clftree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=3, max_leaf_nodes=6,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=23, splitter='best')

In [18]:
# plot the tree
graphClf = print_tree(clftree, features=X.columns, class_names=['No', 'Yes'])
Image(graphClf.create_png())

InvocationException: GraphViz's executables not found

In [20]:
# Training Set Accuracy
print("Training Set Classification Accuracy:", clftree.score(X_train,y_train))

# confusion matrix
print("Confusion Matrix:")
cm = pd.DataFrame(confusion_matrix(y_train, clftree.predict(X_train)), index=['No', 'Yes'], columns=['No', 'Yes'])
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
display(cm)

# Classification Report
print("Classification Report:")
print(classification_report(y_train, clftree.predict(X_train)))

Training Set Classification Accuracy: 0.8040540540540541
Confusion Matrix:


Predicted,No,Yes
Actual,,
No,61,16
Yes,13,58


Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.79      0.81        77
           1       0.78      0.82      0.80        71

    accuracy                           0.80       148
   macro avg       0.80      0.80      0.80       148
weighted avg       0.80      0.80      0.80       148



In [21]:
# Test Set Accuracy
print("Test Set Classification Accuracy:", clftree.score(X_test,y_test))

# confusion matrix
print("Confusion Matrix:")
cm = pd.DataFrame(confusion_matrix(y_test, clftree.predict(X_test)), index=['No', 'Yes'], columns=['No', 'Yes'])
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
display(cm)

# Classification Report
print("Classification Report:")
print(classification_report(y_test, clftree.predict(X_test)))

Test Set Classification Accuracy: 0.7583892617449665
Confusion Matrix:


Predicted,No,Yes
Actual,,
No,58,25
Yes,11,55


Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.70      0.76        83
           1       0.69      0.83      0.75        66

    accuracy                           0.76       149
   macro avg       0.76      0.77      0.76       149
weighted avg       0.77      0.76      0.76       149



### Regression Trees
Decision Trees are also capable of performing regression tasks. <br>We can use Scikit-Learn's `DecisionTreeRegressor` to produce regression trees.

### Hitters Dataset
We use the **Hitters.csv** for our discussion on Regression Trees.

In [ ]:
hitters = pd.read_csv('Hitters.csv').drop('Unnamed: 0', axis=1).dropna()
hitters.head()

In [ ]:
# Factorizing Categoricals
hitters.League = pd.factorize(hitters.League)[0]
hitters.Division = pd.factorize(hitters.Division)[0]
hitters.NewLeague=pd.factorize(hitters.NewLeague)[0]

Let's predict a baseball player's salary based on `Years` (the number of years that he has played in the major leagues) and `Hits` (the number of hits that he made in the previous year).

In [ ]:
X1 = hitters[['Years', 'Hits']].as_matrix()
y1 = np.log(hitters.Salary.as_matrix()) # log transformation

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11,4))

ax1.hist(hitters.Salary.as_matrix())
ax1.set_xlabel('Salary')
ax1.set_ylabel('Count')

ax2.hist(y1)
ax2.set_xlabel('Log(Salary)')
ax2.set_ylabel('Count');

In [ ]:
#documentation: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
regtree = DecisionTreeRegressor(max_leaf_nodes=3, random_state=23)
regtree.fit(X1, y1)

In [ ]:
graphRegTree = print_tree(regtree, features=['Years', 'Hits'])
Image(graphRegTree.create_png())

In [ ]:
# we can plot the decision tree regions on the data
hitters.plot('Years', 'Hits', kind='scatter', color='orange', figsize=(7,6))
plt.xlim(0,25)
plt.ylim(ymin=-5)
plt.xticks([1, 4.5, 24])
plt.yticks([1, 117.5, 238])
plt.vlines(4.5, ymin=-5, ymax=250)
plt.hlines(117.5, xmin=4.5, xmax=25)
plt.annotate('R1', xy=(2,117.5), fontsize='xx-large')
plt.annotate('R2', xy=(11,60), fontsize='xx-large')
plt.annotate('R3', xy=(11,170), fontsize='xx-large');

In [ ]:
hitters_X = hitters.drop('Salary', axis=1) 
hitters_X.head()

In [ ]:
# Training Test Split
X2 = hitters_X.as_matrix()
y2 = np.log(hitters.Salary.as_matrix())
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.5, random_state=23)

In [ ]:
regtree2 = DecisionTreeRegressor(max_depth=3, random_state=23)
regtree2.fit(X2_train, y2_train)
pred = regtree2.predict(X2_test)

In [ ]:
graph2 = print_tree(regtree2, features=hitters_X.columns)
Image(graph2.create_png())

In [ ]:
# Plotting predicted vs. actual
plt.scatter(pred, y2_test, label='Salary')
plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
plt.xlabel('pred')
plt.ylabel('y2_test');

In [ ]:
mean_squared_error(y2_test, pred)

### Bagging and Random Forest

The decision trees we discussed suffer from high variance. <br>

**Bagging**: Bootstrap aggregation, or bagging, is a general-purpose procedure for reducing the variance of a statistical learning method.

Averaging a set of observations reduces variance. Hence a natural way to reduce the variance and hence increase the prediction accuracy of a statistical learning method is to take many training sets from the population, build a separate prediction model using each training set, and average the resulting predictions.

In the classification situation, there are a few possible approaches, but the simplest is as follows. For a given test observation, we can record the class predicted by each of the trees, and take a majority vote: the overall prediction is the most commonly occurring majority vote class among the predictions.

**Random Forest**: As in bagging, we build a number of decision trees on bootstrapped training samples. But when building these decision trees, each time a split in a tree is considered, only a random sample of predictors is chosen as split candidates from the full set of predictors.

In [ ]:
#checking the number of features available in the dataset (which is 19)
hitters_X.shape

In [ ]:
# Bagging: that is, we use all features in the dataset
#documentation: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
regtreeBag = RandomForestRegressor(max_features=19, random_state=23)
regtreeBag.fit(X2_train, y2_train)

In [ ]:
predBag = regtreeBag.predict(X2_test)
plt.scatter(predBag, y2_test, label='Salary')
plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
plt.xlabel('predBag')
plt.ylabel('y2_test');

In [ ]:
mean_squared_error(y2_test, predBag)

In [ ]:
# Random forests: using 6 features
regtreeRF = RandomForestRegressor(max_features=6, random_state=23)
regtreeRF.fit(X2_train, y2_train)

In [ ]:
predRF = regtreeRF.predict(X2_test)
plt.scatter(predRF, y2_test)
plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
plt.xlabel('predRF')
plt.ylabel('y2_test')

In [ ]:
mean_squared_error(y2_test, predRF)

In [ ]:
Importance = pd.DataFrame({'Importance':regtreeRF.feature_importances_*100}, index=hitters_X.columns)
Importance.sort_values('Importance', axis=0, ascending=True).plot(kind='barh', color='r', width = 0.8)
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

### Boosting
A key diﬀerence between boosting and random forests: in boosting, the growth of a particular tree takes into account the other trees that have already been grown. This often results in using smaller trees, which aids interpretability of the model.

In [ ]:
# documentation: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
regtreeBoost = GradientBoostingRegressor(n_estimators=500, learning_rate=0.01, random_state=23)
regtreeBoost.fit(X2_train, y2_train)

In [ ]:
predBoost = regtreeBoost.predict(X2_test)
plt.scatter(predBoost, y2_test)
plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
plt.xlabel('predBoost')
plt.ylabel('y2_test')

In [ ]:
mean_squared_error(y2_test, predBoost)

In [ ]:
feature_importance = regtreeBoost.feature_importances_*100
rel_imp = pd.Series(feature_importance, index=hitters_X.columns).sort_values(inplace=False)
print(rel_imp)
rel_imp.T.plot(kind='barh', color='r', width=0.8)
plt.xlabel('Variable Importance')
plt.gca().legend_ = None